In [26]:
### Mixture density network to predict EURUSD currency.
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers

In [27]:
# Read the excel file
eurusd = pd.read_excel('C:/Users/lenovo/Desktop/currency.xlsx',sheet_name = "EURUSD")
eurusd["price_t-1"] = eurusd['Last Price'].shift(1)
eurusd

,Date,Last Price,price_t-1
0,2022-06-27 22:00:00,1.0585,NaN
1,2022-06-27 22:01:00,1.0581,1.0585
2,2022-06-27 22:02:00,1.0584,1.0581
3,2022-06-27 22:03:00,1.0584,1.0584
4,2022-06-27 22:04:00,1.0584,1.0584
...,...,...,...
12847,2022-07-08 18:15:00,1.0171,1.0172
12848,2022-07-08 18:16:00,1.0171,1.0171
12849,2022-07-08 18:17:00,1.0172,1.0171
12850,2022-07-08 18:18:00,1.0170,1.0172


In [28]:
# Calculate the return at time t
eurusd['return_t'] = np.log(eurusd['Last Price']) - np.log(eurusd["price_t-1"])
eurusd

,Date,Last Price,price_t-1,return_t
0,2022-06-27 22:00:00,1.0585,NaN,NaN
1,2022-06-27 22:01:00,1.0581,1.0585,-0.000378
2,2022-06-27 22:02:00,1.0584,1.0581,0.000283
3,2022-06-27 22:03:00,1.0584,1.0584,0.000000
4,2022-06-27 22:04:00,1.0584,1.0584,0.000000
...,...,...,...,...
12847,2022-07-08 18:15:00,1.0171,1.0172,-0.000098
12848,2022-07-08 18:16:00,1.0171,1.0171,0.000000
12849,2022-07-08 18:17:00,1.0172,1.0171,0.000098
12850,2022-07-08 18:18:00,1.0170,1.0172,-0.000197


In [29]:
# Use data in the late 10 minutes as features
window = 10
for i in range(1,window + 1):
    string = 'return_t-' + str(i)
    eurusd[string] = eurusd['return_t'].shift(i)
eurusd

,Date,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10
0,2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12847,2022-07-08 18:15:00,1.0171,1.0172,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,0.000197
12848,2022-07-08 18:16:00,1.0171,1.0171,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197
12849,2022-07-08 18:17:00,1.0172,1.0171,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000
12850,2022-07-08 18:18:00,1.0170,1.0172,-0.000197,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000


In [30]:
# Set data as index
eurusd.set_index('Date',inplace=True)
eurusd

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10
Date,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 18:15:00,1.0171,1.0172,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,0.000197
2022-07-08 18:16:00,1.0171,1.0171,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197
2022-07-08 18:17:00,1.0172,1.0171,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000


In [31]:
# Add S&P500 as feature
spx = pd.read_excel('C:/Users/lenovo/Desktop/indices.xlsx',sheet_name = "S&P500")
spx.set_index('Date',inplace=True)
spx.rename(columns={'Last Price':'SPX'},inplace=True)
spx = spx['SPX']
eurusd_spx = eurusd.merge(spx, how='outer', left_index=True, right_index=True)
eurusd_spx

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX
Date,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 18:15:00,1.0171,1.0172,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,0.000197,3889.08
2022-07-08 18:16:00,1.0171,1.0171,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,3889.09
2022-07-08 18:17:00,1.0172,1.0171,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,3888.80


In [32]:
# Add Euro Stoxx50 as feature
sx5e = pd.read_excel('C:/Users/lenovo/Desktop/indices.xlsx',sheet_name = "Euro Stoxx50")
sx5e.set_index('Date',inplace=True)
sx5e.rename(columns={'Last Price':'SX5E'},inplace=True)
sx5e = sx5e['SX5E']
eurusd_spx_sx5e = eurusd_spx.merge(sx5e, how='outer', left_index=True, right_index=True)
eurusd_spx_sx5e

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E
Date,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 18:15:00,1.0171,1.0172,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,0.000197,3889.08,NaN
2022-07-08 18:16:00,1.0171,1.0171,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,0.000197,3889.09,NaN
2022-07-08 18:17:00,1.0172,1.0171,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,0.000000,0.000197,0.000098,0.000000,0.000000,3888.80,NaN


In [33]:
# Add implied volatility as feature
vol = pd.read_excel('C:/Users/lenovo/Desktop/implied_vol.xlsx',sheet_name = "EURUSDV1M")
vol.set_index('Date',inplace=True)
vol.rename(columns={'Last Price':'VOL'},inplace=True)
vol = vol['VOL']
eurusd_spx_sx5e_vol = eurusd_spx_sx5e.merge(vol, how='outer', left_index=True, right_index=True)
eurusd_spx_sx5e_vol

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL
Date,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 18:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7625
2022-07-08 18:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7625
2022-07-08 18:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7625


In [34]:
# Add interest rate as feature

eusa10 = pd.read_excel('C:/Users/lenovo/Desktop/interest_rate.xlsx',sheet_name = "EUSA10")
eusa10.set_index('Date',inplace=True)
eusa10.rename(columns={'Last Price':'EUSA10'},inplace=True)
eusa10 = eusa10['EUSA10']
eurusd_spx_sx5e_vol_eusa10 = eurusd_spx_sx5e_vol.merge(eusa10, how='outer', left_index=True, right_index=True)
eurusd_spx_sx5e_vol_eusa10

ussw10 = pd.read_excel('C:/Users/lenovo/Desktop/interest_rate.xlsx',sheet_name = "USSW10")
ussw10.set_index('Date',inplace=True)
ussw10.rename(columns={'Last Price':'USSW10'},inplace=True)
ussw10 = ussw10['USSW10']
eurusd_spx_sx5e_vol_eusa10_ussw10 = eurusd_spx_sx5e_vol_eusa10.merge(ussw10, how='outer', left_index=True, right_index=True)
eurusd_com = eurusd_spx_sx5e_vol_eusa10_ussw10
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
Date,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2755
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2760
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2747
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025,NaN,3.2740
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1863
2022-07-08 19:11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1850
2022-07-08 19:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1862


In [35]:
# Fill NAN
eurusd_com = eurusd_com.fillna(method ='ffill')
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SPX,SX5E,VOL,EUSA10,USSW10
Date,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2755
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2760
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2747
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025,NaN,3.2740
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025,NaN,3.2740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3887.84,3506.55,10.7625,2.156,3.1863
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3887.84,3506.55,10.7625,2.156,3.1850
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3887.84,3506.55,10.7625,2.156,3.1862


In [38]:
# Calculate SPX return, and add 3 minutes lagged return
eurusd_com['SPX_t-1'] = eurusd_com['SPX'].shift(1)
eurusd_com['SPX_return_t'] = np.log(eurusd_com['SPX']) - np.log(eurusd_com["SPX_t-1"])
window = 3
for i in range(1,window + 1):
    string = 'SPX_return_t-' + str(i)
    eurusd_com[string] = eurusd_com['SPX_return_t'].shift(i)
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,SPX,SX5E,VOL,EUSA10,USSW10,SPX_t-1,SPX_return_t,SPX_return_t-1,SPX_return_t-2,SPX_return_t-3
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.3850,NaN,3.2755,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.3850,NaN,3.2760,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.3850,NaN,3.2747,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.4025,NaN,3.2740,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,...,NaN,NaN,8.4025,NaN,3.2740,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,3887.84,3506.55,10.7625,2.156,3.1863,3887.84,0.0,0.0,0.0,0.0
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,3887.84,3506.55,10.7625,2.156,3.1850,3887.84,0.0,0.0,0.0,0.0
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,3887.84,3506.55,10.7625,2.156,3.1862,3887.84,0.0,0.0,0.0,0.0


In [39]:
# Delete redundant columns
del eurusd_com['SPX'], eurusd_com['SPX_t-1'],eurusd_com['SPX_return_t']

In [40]:
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,SX5E,VOL,EUSA10,USSW10,SPX_return_t-1,SPX_return_t-2,SPX_return_t-3
Date,,,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2755,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2760,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.3850,NaN,3.2747,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025,NaN,3.2740,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4025,NaN,3.2740,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3506.55,10.7625,2.156,3.1863,0.0,0.0,0.0
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3506.55,10.7625,2.156,3.1850,0.0,0.0,0.0
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,0.0,0.000197,0.000098,3506.55,10.7625,2.156,3.1862,0.0,0.0,0.0


In [41]:
# Same method to calculate SX5E return
eurusd_com['SX5E_t-1'] = eurusd_com['SX5E'].shift(1)
eurusd_com['SX5E_return_t'] = np.log(eurusd_com['SX5E']) - np.log(eurusd_com["SX5E_t-1"])
window = 3
for i in range(1,window + 1):
    string = 'SX5E_return_t-' + str(i)
    eurusd_com[string] = eurusd_com['SX5E_return_t'].shift(i)
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,EUSA10,USSW10,SPX_return_t-1,SPX_return_t-2,SPX_return_t-3,SX5E_t-1,SX5E_return_t,SX5E_return_t-1,SX5E_return_t-2,SX5E_return_t-3
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.2755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.2760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.2747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,...,NaN,3.2740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,...,NaN,3.2740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,2.156,3.1863,0.0,0.0,0.0,3506.55,0.0,0.0,0.0,0.0
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,2.156,3.1850,0.0,0.0,0.0,3506.55,0.0,0.0,0.0,0.0
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,2.156,3.1862,0.0,0.0,0.0,3506.55,0.0,0.0,0.0,0.0


In [42]:
# VOL return
eurusd_com['VOL_t-1'] = eurusd_com['VOL'].shift(1)
eurusd_com['VOL_return_t'] = np.log(eurusd_com['VOL']) - np.log(eurusd_com["VOL_t-1"])
window = 3
for i in range(1,window + 1):
    string = 'VOL_return_t-' + str(i)
    eurusd_com[string] = eurusd_com['VOL_return_t'].shift(i)
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,SX5E_t-1,SX5E_return_t,SX5E_return_t-1,SX5E_return_t-2,SX5E_return_t-3,VOL_t-1,VOL_return_t,VOL_return_t-1,VOL_return_t-2,VOL_return_t-3
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.3850,0.000000,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.3850,0.000000,0.000000,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.3850,0.002085,0.000000,0.0,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.4025,0.000000,0.002085,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,3506.55,0.0,0.0,0.0,0.0,10.7625,0.000000,0.000000,0.0,0.0
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,3506.55,0.0,0.0,0.0,0.0,10.7625,0.000000,0.000000,0.0,0.0
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,3506.55,0.0,0.0,0.0,0.0,10.7625,0.000000,0.000000,0.0,0.0


In [43]:
# EUSA10 return
eurusd_com['EUSA10_t-1'] = eurusd_com['EUSA10'].shift(1)
eurusd_com['EUSA10_return_t'] = np.log(eurusd_com['EUSA10']) - np.log(eurusd_com["EUSA10_t-1"])
window = 3
for i in range(1,window + 1):
    string = 'EUSA10_return_t-' + str(i)
    eurusd_com[string] = eurusd_com['EUSA10_return_t'].shift(i)
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,VOL_t-1,VOL_return_t,VOL_return_t-1,VOL_return_t-2,VOL_return_t-3,EUSA10_t-1,EUSA10_return_t,EUSA10_return_t-1,EUSA10_return_t-2,EUSA10_return_t-3
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.3850,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,...,8.3850,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,...,8.3850,0.002085,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,...,8.4025,0.000000,0.002085,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,10.7625,0.000000,0.000000,0.0,0.0,2.156,0.0,0.0,0.0,0.0
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,10.7625,0.000000,0.000000,0.0,0.0,2.156,0.0,0.0,0.0,0.0
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,10.7625,0.000000,0.000000,0.0,0.0,2.156,0.0,0.0,0.0,0.0


In [44]:
# USSW10 return
eurusd_com['USSW10_t-1'] = eurusd_com['USSW10'].shift(1)
eurusd_com['USSW10_return_t'] = np.log(eurusd_com['USSW10']) - np.log(eurusd_com["USSW10_t-1"])
window = 3
for i in range(1,window + 1):
    string = 'USSW10_return_t-' + str(i)
    eurusd_com[string] = eurusd_com['USSW10_return_t'].shift(i)
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,EUSA10_t-1,EUSA10_return_t,EUSA10_return_t-1,EUSA10_return_t-2,EUSA10_return_t-3,USSW10_t-1,USSW10_return_t,USSW10_return_t-1,USSW10_return_t-2,USSW10_return_t-3
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-27 22:00:00,1.0585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27 22:01:00,1.0581,1.0585,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.2755,0.000153,NaN,NaN,NaN
2022-06-27 22:02:00,1.0584,1.0581,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.2760,-0.000397,0.000153,NaN,NaN
2022-06-27 22:03:00,1.0584,1.0584,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.2747,-0.000214,-0.000397,0.000153,NaN
2022-06-27 22:04:00,1.0584,1.0584,0.000000,0.000000,0.000283,-0.000378,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.2740,0.000000,-0.000214,-0.000397,0.000153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,2.156,0.0,0.0,0.0,0.0,3.1864,-0.000031,0.000063,0.000377,-0.000314
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,2.156,0.0,0.0,0.0,0.0,3.1863,-0.000408,-0.000031,0.000063,0.000377
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.0,0.000098,0.0,...,2.156,0.0,0.0,0.0,0.0,3.1850,0.000377,-0.000408,-0.000031,0.000063


In [45]:
# Delete redundant columns
del eurusd_com['SX5E'], eurusd_com['SX5E_t-1'],eurusd_com['SX5E_return_t']
del eurusd_com['VOL'], eurusd_com['VOL_t-1'],eurusd_com['VOL_return_t']
del eurusd_com['EUSA10'], eurusd_com['EUSA10_t-1'],eurusd_com['EUSA10_return_t']
del eurusd_com['USSW10'], eurusd_com['USSW10_t-1'],eurusd_com['USSW10_return_t']

In [47]:
eurusd_com.columns

Index(['Last Price', 'price_t-1', 'return_t', 'return_t-1', 'return_t-2',
       'return_t-3', 'return_t-4', 'return_t-5', 'return_t-6', 'return_t-7',
       'return_t-8', 'return_t-9', 'return_t-10', 'SPX_return_t-1',
       'SPX_return_t-2', 'SPX_return_t-3', 'SX5E_return_t-1',
       'SX5E_return_t-2', 'SX5E_return_t-3', 'VOL_return_t-1',
       'VOL_return_t-2', 'VOL_return_t-3', 'EUSA10_return_t-1',
       'EUSA10_return_t-2', 'EUSA10_return_t-3', 'USSW10_return_t-1',
       'USSW10_return_t-2', 'USSW10_return_t-3'],
      dtype='object')

In [48]:
# Find the first row that does not contain NAN. Delete all data before this row.

In [49]:
notna_index = eurusd_com[eurusd_com.notna().all(axis = 1)].index[0]
eurusd_com = eurusd_com.loc[notna_index : ,:]
eurusd_com

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,SX5E_return_t-3,VOL_return_t-1,VOL_return_t-2,VOL_return_t-3,EUSA10_return_t-1,EUSA10_return_t-2,EUSA10_return_t-3,USSW10_return_t-1,USSW10_return_t-2,USSW10_return_t-3
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-28 14:34:00,1.0531,1.0528,0.000285,-0.000190,0.000000,-0.000285,0.000285,-0.000285,0.000095,0.000000,...,0.000426,0.0,0.001167,0.000000,0.000537,0.004056,-0.003313,0.000121,0.001973,-0.001882
2022-06-28 14:35:00,1.0531,1.0531,0.000000,0.000285,-0.000190,0.000000,-0.000285,0.000285,-0.000285,0.000095,...,0.001810,0.0,0.000000,0.001167,-0.000867,0.000537,0.004056,-0.002307,0.000121,0.001973
2022-06-28 14:36:00,1.0530,1.0531,-0.000095,0.000000,0.000285,-0.000190,0.000000,-0.000285,0.000285,-0.000285,...,-0.000106,0.0,0.000000,0.000000,-0.000868,-0.000867,0.000537,-0.000456,-0.002307,0.000121
2022-06-28 14:37:00,1.0530,1.0530,0.000000,-0.000095,0.000000,0.000285,-0.000190,0.000000,-0.000285,0.000285,...,0.000020,0.0,0.000000,0.000000,0.000496,-0.000868,-0.000867,0.001458,-0.000456,-0.002307
2022-06-28 14:38:00,1.0530,1.0530,0.000000,0.000000,-0.000095,0.000000,0.000285,-0.000190,0.000000,-0.000285,...,0.000531,0.0,0.000000,0.000000,-0.001696,0.000496,-0.000868,-0.000820,0.001458,-0.000456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-08 19:10:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000063,0.000377,-0.000314
2022-07-08 19:11:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000031,0.000063,0.000377
2022-07-08 19:12:00,1.0170,1.0170,0.000000,-0.000197,0.000098,0.000000,-0.000098,0.000000,0.000098,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000408,-0.000031,0.000063


In [50]:
# Split training set and testing set. The testing set accounts for 10% of the whole data.
num = eurusd_com.shape[0]//10
eurusd_train = eurusd_com.iloc[:-num,:]
eurusd_test = eurusd_com.iloc[-num: ,]

In [51]:
eurusd_train

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,SX5E_return_t-3,VOL_return_t-1,VOL_return_t-2,VOL_return_t-3,EUSA10_return_t-1,EUSA10_return_t-2,EUSA10_return_t-3,USSW10_return_t-1,USSW10_return_t-2,USSW10_return_t-3
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-28 14:34:00,1.0531,1.0528,0.000285,-0.000190,0.000000,-0.000285,0.000285,-0.000285,0.000095,0.000000,...,0.000426,0.000000,0.001167,0.000000,0.000537,0.004056,-0.003313,0.000121,0.001973,-0.001882
2022-06-28 14:35:00,1.0531,1.0531,0.000000,0.000285,-0.000190,0.000000,-0.000285,0.000285,-0.000285,0.000095,...,0.001810,0.000000,0.000000,0.001167,-0.000867,0.000537,0.004056,-0.002307,0.000121,0.001973
2022-06-28 14:36:00,1.0530,1.0531,-0.000095,0.000000,0.000285,-0.000190,0.000000,-0.000285,0.000285,-0.000285,...,-0.000106,0.000000,0.000000,0.000000,-0.000868,-0.000867,0.000537,-0.000456,-0.002307,0.000121
2022-06-28 14:37:00,1.0530,1.0530,0.000000,-0.000095,0.000000,0.000285,-0.000190,0.000000,-0.000285,0.000285,...,0.000020,0.000000,0.000000,0.000000,0.000496,-0.000868,-0.000867,0.001458,-0.000456,-0.002307
2022-06-28 14:38:00,1.0530,1.0530,0.000000,0.000000,-0.000095,0.000000,0.000285,-0.000190,0.000000,-0.000285,...,0.000531,0.000000,0.000000,0.000000,-0.001696,0.000496,-0.000868,-0.000820,0.001458,-0.000456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-07 23:18:00,1.0163,1.0163,0.000000,0.000000,-0.000098,-0.000197,0.000000,0.000000,-0.000098,0.000000,...,0.000000,0.000233,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-07-07 23:19:00,1.0164,1.0163,0.000098,0.000000,0.000000,-0.000098,-0.000197,0.000000,0.000000,-0.000098,...,0.000000,0.000000,0.000233,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-07-07 23:20:00,1.0163,1.0164,-0.000098,0.000098,0.000000,0.000000,-0.000098,-0.000197,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000233,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [52]:
# Nolmallize data by minmax scaler
from sklearn import preprocessing
# Construct a scaler
MiMaScaler = preprocessing.MinMaxScaler()
# Fit data on training set
MiMaScaler.fit(eurusd_train)
# Generate the scaled data
train = MiMaScaler.transform(eurusd_train)
scaled_eurusd_train = pd.DataFrame(train, columns=eurusd_train.columns,index = eurusd_train.index)
scaled_eurusd_train.describe()

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,SX5E_return_t-3,VOL_return_t-1,VOL_return_t-2,VOL_return_t-3,EUSA10_return_t-1,EUSA10_return_t-2,EUSA10_return_t-3,USSW10_return_t-1,USSW10_return_t-2,USSW10_return_t-3
count,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,...,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000,10735.000000
mean,0.579229,0.579312,0.374061,0.374087,0.373991,0.374020,0.374024,0.374017,0.374019,0.374055,...,0.525998,0.501656,0.501658,0.501658,0.499837,0.499842,0.499838,0.349349,0.349359,0.349350
std,0.305832,0.305810,0.044031,0.044062,0.044031,0.044028,0.044033,0.044039,0.044039,0.044049,...,0.014768,0.013312,0.013313,0.013313,0.018757,0.018765,0.018770,0.037823,0.037836,0.037848
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.288608,0.288608,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,0.349840,...,0.526073,0.501424,0.501424,0.501424,0.500000,0.500000,0.500000,0.337387,0.337387,0.337383
50%,0.716456,0.716456,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,...,0.526073,0.501424,0.501424,0.501424,0.500000,0.500000,0.500000,0.349676,0.349676,0.349676
75%,0.787342,0.787342,0.399926,0.399926,0.399923,0.399923,0.399923,0.399923,0.399923,0.399926,...,0.526073,0.501424,0.501424,0.501424,0.500000,0.500000,0.500000,0.361360,0.361386,0.361386
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [54]:
# Scale testing set by using this scaler
test = MiMaScaler.transform(eurusd_test)
scaled_eurusd_test = pd.DataFrame(test, columns=eurusd_test.columns,index = eurusd_test.index)
scaled_eurusd_test.describe()

,Last Price,price_t-1,return_t,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,...,SX5E_return_t-3,VOL_return_t-1,VOL_return_t-2,VOL_return_t-3,EUSA10_return_t-1,EUSA10_return_t-2,EUSA10_return_t-3,USSW10_return_t-1,USSW10_return_t-2,USSW10_return_t-3
count,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,...,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000,1192.000000
mean,0.024562,0.024548,0.375039,0.372689,0.376268,0.375039,0.373875,0.375082,0.376268,0.375039,...,0.526227,0.501470,0.501470,0.501470,0.500150,0.500150,0.500150,0.351118,0.351116,0.351100
std,0.057470,0.057462,0.062249,0.063188,0.062470,0.062240,0.062481,0.062240,0.062470,0.062240,...,0.010889,0.009488,0.009488,0.009488,0.017510,0.017510,0.017510,0.046288,0.046288,0.046285
min,-0.169620,-0.169620,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,-0.346741,...,0.444766,0.423322,0.423322,0.423322,0.351950,0.351950,0.351950,0.174646,0.174646,0.174646
25%,-0.002532,-0.002532,0.349165,0.349122,0.349165,0.349165,0.349165,0.349165,0.349165,0.349165,...,0.526073,0.501424,0.501424,0.501424,0.500000,0.500000,0.500000,0.333277,0.333277,0.333277
50%,0.040506,0.040506,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,0.374888,...,0.526073,0.501424,0.501424,0.501424,0.500000,0.500000,0.500000,0.349676,0.349676,0.349676
75%,0.063291,0.063291,0.400626,0.400626,0.400626,0.400626,0.400626,0.400626,0.400626,0.400626,...,0.526073,0.501424,0.501424,0.501424,0.500000,0.500000,0.500000,0.368301,0.368301,0.368294
max,0.116456,0.116456,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,0.864444,...,0.580553,0.635587,0.635587,0.635587,0.823293,0.823293,0.823293,1.340632,1.340632,1.340632


In [55]:
# Split label and features
# For training set
train_label = scaled_eurusd_train[['return_t']]
train_label

,return_t
Date,
2022-06-28 14:34:00,0.449360
2022-06-28 14:35:00,0.374888
2022-06-28 14:36:00,0.350066
2022-06-28 14:37:00,0.374888
2022-06-28 14:38:00,0.374888
...,...
2022-07-07 23:18:00,0.374888
2022-07-07 23:19:00,0.400606
2022-07-07 23:20:00,0.349170


In [56]:
train_feature = scaled_eurusd_train.loc[:,'return_t-1':]
train_feature

,return_t-1,return_t-2,return_t-3,return_t-4,return_t-5,return_t-6,return_t-7,return_t-8,return_t-9,return_t-10,...,SX5E_return_t-3,VOL_return_t-1,VOL_return_t-2,VOL_return_t-3,EUSA10_return_t-1,EUSA10_return_t-2,EUSA10_return_t-3,USSW10_return_t-1,USSW10_return_t-2,USSW10_return_t-3
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-28 14:34:00,0.325238,0.374888,0.300430,0.449346,0.300430,0.399705,0.374888,0.374888,0.350071,0.350073,...,0.541137,0.501424,0.514499,0.501424,0.507269,0.554924,0.455147,0.356001,0.452558,0.251538
2022-06-28 14:35:00,0.449360,0.325238,0.374888,0.300430,0.449346,0.300430,0.399705,0.374888,0.374888,0.350071,...,0.590121,0.501424,0.501424,0.514499,0.488256,0.507269,0.554924,0.229378,0.356001,0.452558
2022-06-28 14:36:00,0.374888,0.449360,0.325238,0.374888,0.300430,0.449346,0.300430,0.399705,0.374888,0.374888,...,0.522315,0.501424,0.501424,0.501424,0.488245,0.488256,0.507269,0.325900,0.229378,0.356001
2022-06-28 14:37:00,0.350066,0.374888,0.449360,0.325238,0.374888,0.300430,0.449346,0.300430,0.399705,0.374888,...,0.526766,0.501424,0.501424,0.501424,0.506718,0.488245,0.488256,0.425721,0.325900,0.229378
2022-06-28 14:38:00,0.374888,0.350066,0.374888,0.449360,0.325238,0.374888,0.300430,0.449346,0.300430,0.399705,...,0.544861,0.501424,0.501424,0.501424,0.477032,0.506718,0.488245,0.306915,0.425721,0.325900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-07 23:18:00,0.374888,0.349170,0.323460,0.374888,0.374888,0.349178,0.374888,0.374888,0.400598,0.374888,...,0.526073,0.504040,0.501424,0.501424,0.500000,0.500000,0.500000,0.349676,0.349676,0.349676
2022-07-07 23:19:00,0.374888,0.374888,0.349170,0.323460,0.374888,0.374888,0.349178,0.374888,0.374888,0.400598,...,0.526073,0.501424,0.504040,0.501424,0.500000,0.500000,0.500000,0.349676,0.349676,0.349676
2022-07-07 23:20:00,0.400606,0.374888,0.374888,0.349170,0.323460,0.374888,0.374888,0.349178,0.374888,0.374888,...,0.526073,0.501424,0.501424,0.504040,0.500000,0.500000,0.500000,0.349676,0.349676,0.349676


In [57]:
# For testing set
test_label = scaled_eurusd_test[['return_t']]
test_feature = scaled_eurusd_test.loc[:,'return_t-1':]

In [68]:
# Build the model
event_shape = [1]
num_components = 5  # normal mixture with 5 components
params_size = tfp.layers.MixtureNormal.params_size(num_components, event_shape)
model = tfk.Sequential([
    tfkl.Dense(10, activation='relu'),
    tfkl.Dense(12, activation='relu'),
    tfkl.Dense(10, activation='relu'),
    tfkl.Dense(params_size, activation=None),
    tfpl.MixtureNormal(num_components, event_shape)
])

In [69]:
# Fit
model.compile(optimizer="adam",
              loss=lambda y, model: -model.log_prob(y))
model.fit(train_feature, train_label, batch_size=64, epochs=300)

Epoch 1/300
168/168 [==============================] - 1s 1ms/step - loss: -0.8263
Epoch 2/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7390
Epoch 3/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7405
Epoch 4/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7555
Epoch 5/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7678
Epoch 6/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7589
Epoch 7/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7671
Epoch 8/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7689
Epoch 9/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7724
Epoch 10/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7799
Epoch 11/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7885
Epoch 12/300
168/168 [==============================] - 0s 1ms/step - loss: -1.7800
E

168/168 [==============================] - 0s 1ms/step - loss: -1.8479
Epoch 99/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8379
Epoch 100/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8391
Epoch 101/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8417
Epoch 102/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8411
Epoch 103/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8418
Epoch 104/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8407
Epoch 105/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8447
Epoch 106/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8413
Epoch 107/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8434
Epoch 108/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8460
Epoch 109/300
168/168 [==============================] - 0s 1ms/step - loss: -1.

168/168 [==============================] - 0s 1ms/step - loss: -1.8527
Epoch 195/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8520
Epoch 196/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8522
Epoch 197/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8544
Epoch 198/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8485
Epoch 199/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8518
Epoch 200/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8474
Epoch 201/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8536
Epoch 202/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8468
Epoch 203/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8511
Epoch 204/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8575
Epoch 205/300
168/168 [==============================] - 0s 1ms/step - loss: -1

168/168 [==============================] - 0s 1ms/step - loss: -1.8620
Epoch 291/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8614
Epoch 292/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8643
Epoch 293/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8636
Epoch 294/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8619
Epoch 295/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8668
Epoch 296/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8680
Epoch 297/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8646
Epoch 298/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8652
Epoch 299/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8595
Epoch 300/300
168/168 [==============================] - 0s 1ms/step - loss: -1.8659


In [70]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                260       
                                                                 
 dense_5 (Dense)             (None, 12)                132       
                                                                 
 dense_6 (Dense)             (None, 10)                130       
                                                                 
 dense_7 (Dense)             (None, 15)                165       
                                                                 
 mixture_normal_1 (MixtureNo  ((None, 1),              0         
 rmal)                        (None, 1))                         
                                                                 
Total params: 687
Trainable params: 687
Non-trainable params: 0
________________________________________________________

In [71]:
# Create two empty columns
test_label["mix_mean"] = np.nan
test_label["mix_dev"] = np.nan
test_label

<ipython-input-71-1f5e28c17cee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label["mix_mean"] = np.nan
<ipython-input-71-1f5e28c17cee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label["mix_dev"] = np.nan


,return_t,mix_mean,mix_dev,a,b
Date,,,,,
2022-07-07 23:23:00,0.349167,NaN,NaN,0.341244,0.422876
2022-07-07 23:24:00,0.374888,NaN,NaN,0.339227,0.422742
2022-07-07 23:25:00,0.374888,NaN,NaN,0.338166,0.422340
2022-07-07 23:26:00,0.349165,NaN,NaN,0.338486,0.422349
2022-07-07 23:27:00,0.400611,NaN,NaN,0.341145,0.422868
...,...,...,...,...,...
2022-07-08 19:10:00,0.374888,NaN,NaN,0.341798,0.423058
2022-07-08 19:11:00,0.374888,NaN,NaN,0.340065,0.422720
2022-07-08 19:12:00,0.374888,NaN,NaN,0.341073,0.422904


In [72]:
for i in range(test_feature.shape[0]):
    x = np.array(test_feature.iloc[i])
    x = x.reshape(1,x.shape[0])
    gm = model(x)
    gm_weights = gm.mixture_distribution.probs_parameter().numpy().flatten()
    gm_means = gm.components_distribution.mean().numpy().flatten()
    gm_vars = gm.components_distribution.variance().numpy().flatten()
    mix_mean = np.dot(gm_weights, gm_means)
    mix_dev = np.sqrt(np.dot(gm_weights**2, gm_vars))
    test_label.iloc[i,1] = mix_mean
    test_label.iloc[i,2] = mix_dev

D:\anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-72-dc538d6fefb8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label.iloc[i,1] = mix_mean
<ipython-input-72-dc538d6fefb8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label.iloc[i,2] = mix_dev


In [73]:
test_label

,return_t,mix_mean,mix_dev,a,b
Date,,,,,
2022-07-07 23:23:00,0.349167,0.378308,0.017549,0.341244,0.422876
2022-07-07 23:24:00,0.374888,0.380152,0.014536,0.339227,0.422742
2022-07-07 23:25:00,0.374888,0.380455,0.016101,0.338166,0.422340
2022-07-07 23:26:00,0.349165,0.378878,0.015967,0.338486,0.422349
2022-07-07 23:27:00,0.400611,0.379341,0.016243,0.341145,0.422868
...,...,...,...,...,...
2022-07-08 19:10:00,0.374888,0.379824,0.017520,0.341798,0.423058
2022-07-08 19:11:00,0.374888,0.378718,0.018095,0.340065,0.422720
2022-07-08 19:12:00,0.374888,0.378536,0.015988,0.341073,0.422904


In [74]:
test_label['a'] = test_label['mix_mean'] - 1.96 * test_label['mix_dev']
test_label['b'] = test_label['mix_mean'] + 1.96 * test_label['mix_dev']

<ipython-input-74-f2e50151ce47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label['a'] = test_label['mix_mean'] - 1.96 * test_label['mix_dev']
<ipython-input-74-f2e50151ce47>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_label['b'] = test_label['mix_mean'] + 1.96 * test_label['mix_dev']


In [75]:
test_label

,return_t,mix_mean,mix_dev,a,b
Date,,,,,
2022-07-07 23:23:00,0.349167,0.378308,0.017549,0.343912,0.412703
2022-07-07 23:24:00,0.374888,0.380152,0.014536,0.351662,0.408642
2022-07-07 23:25:00,0.374888,0.380455,0.016101,0.348897,0.412013
2022-07-07 23:26:00,0.349165,0.378878,0.015967,0.347583,0.410174
2022-07-07 23:27:00,0.400611,0.379341,0.016243,0.347504,0.411178
...,...,...,...,...,...
2022-07-08 19:10:00,0.374888,0.379824,0.017520,0.345486,0.414163
2022-07-08 19:11:00,0.374888,0.378718,0.018095,0.343251,0.414184
2022-07-08 19:12:00,0.374888,0.378536,0.015988,0.347201,0.409872


In [76]:
# See if the return_t falls in the confidence interval
result = (test_label['return_t'] > test_label['a']) & (test_label['return_t'] < test_label['b'])
result.value_counts()

True     838
False    354
dtype: int64

In [77]:
# Calculate the accuracy
accuracy = (result.value_counts().loc[True])/(test_label.shape[0])
accuracy

0.7030201342281879